In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/huggingface-bert/bert-base-multilingual-cased/config.json
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/tokenizer.json
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/tf_model.h5
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/pytorch_model.bin
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/modelcard.json
/kaggle/input/huggingface-bert/bert-base-multilingual-cased/vocab.txt
/kaggle/input/huggingface-bert/bert-large-uncased/config.json
/kaggle/input/huggingface-bert/bert-large-uncased/tokenizer.json
/kaggle/input/huggingface-bert/bert-large-uncased/tf_model.h5
/kaggle/input/huggingface-bert/bert-large-uncased/pytorch_model.bin
/kaggle/input/huggingface-bert/bert-large-uncased/modelcard.json
/kaggle/input/huggingface-bert/bert-large-uncased/vocab.txt
/kaggle/input/huggingface-bert/bert-large-uncased/whole-word-masking/._bert_config.json
/kaggle/input/huggingface-bert/bert-large-uncased/whole-word-masking/bert_config.json
/k

# Сиды

In [2]:
import os
import random
import numpy as np
import torch
import numpy as np # linear algebra
import pandas as pd 
try:
    # Hugging Face convenience fn; sets Python/Rand, NumPy, Torch seeds
    from transformers import set_seed  
except ImportError:
    set_seed = None

def seed_everything(seed):
    # 1. Python built‑ins
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)

    # 2. NumPy
    np.random.seed(seed)

    # 3. PyTorch (CPU & GPU)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # 4. CuDNN: make deterministic, but may slow you down
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # 5. Transformers (if installed)
    if set_seed is not None:
        set_seed(seed)

# call it!
seed_everything(257)

2025-08-04 23:32:38.859759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754350359.037162      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754350359.092213      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# импорты

In [3]:
import torch
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split 
import re

# Загрузка данных

In [4]:
train=pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test=pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
sample=pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')

# EDA

In [5]:
train

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2829,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2830,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2831,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2832,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


In [6]:
test

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...
5,12537fe78,NaN,NaN,"To explain transitivity, let us look first at ..."
6,965e592c0,https://www.africanstorybook.org/#,CC BY 4.0,Milka and John are playing in the garden. Her ...


In [7]:
sample

,id,target
0,c0f722661,0.0
1,f0953f0a5,0.0
2,0df072751,0.0
3,04caf4e0c,0.0
4,0e63f8bea,0.0
5,12537fe78,0.0
6,965e592c0,0.0


# Чистка данных

In [8]:
import re, html, unicodedata

RE_URL   = re.compile(r'https?://\S+|www\.\S+')
RE_EMAIL = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w+\b')
RE_PHONE = re.compile(r'\+?\d[\d\-\s()]{7,}\d')

# Юникодные кавычки/тире/многоточия → ASCII
PUNCT_MAP = {
    '\u2018':'\'', '\u2019':'\'', '\u201C':'"', '\u201D':'"',
    '\u2013':'-', '\u2014':'-', '\u2212':'-', '\u2026':'...',
}

def _normalize_punct(s: str) -> str:
    return s.translate(str.maketrans(PUNCT_MAP))

def clean_for_readability(text: str, keep_case: bool = True) -> str:
    """Бережная чистка: сохраняем пунктуацию/цифры/дефисы и границы предложений."""
    if not isinstance(text, str):
        return ""
    # Юникод + HTML
    s = unicodedata.normalize('NFKC', text)
    s = html.unescape(s)
    s = _normalize_punct(s)

    # Плейсхолдеры сущностей (оставляем их как маркеры сложности)
    s = RE_URL.sub(' <URL> ', s)
    s = RE_EMAIL.sub(' <EMAIL> ', s)
    s = RE_PHONE.sub(' <PHONE> ', s)

    # Убираем управляющие символы, но сохраняем . , ; : ! ? - ' "
    # Заменяем прочие символы на пробел
    s = re.sub(r"[^A-Za-z0-9\s\.\,\;\:\!\?\'\"\-\(\)]", " ", s)

    # Схлопываем повторяющиеся знаки, но оставляем максимум 2 (например ??, --)
    s = re.sub(r'([\.!\?,;:\-])\1{2,}', r'\1\1', s)

    # Правим пробелы вокруг пунктуации
    s = re.sub(r'\s+([.,;:!?])', r'\1', s)
    s = re.sub(r'([(\-])\s+', r'\1', s)     # после ( и -
    s = re.sub(r'\s{2,}', ' ', s).strip()

    if not keep_case:
        s = s.lower()
    return s


In [9]:
#texts_train=train['excerpt'].apply(clean_for_readability)

In [10]:
texts_train=train['excerpt']

In [11]:
texts_train

0       When the young people returned to the ballroom...
1       All through dinner time, Mrs. Fayre was somewh...
2       As Roger had predicted, the snow departed as q...
3       And outside before the palace a great garden w...
4       Once upon a time there were Three Bears who li...
                              ...                        
2829    When you think of dinosaurs and where they liv...
2830    So what is a solid? Solids are usually hard be...
2831    The second state of matter we will discuss is ...
2832    Solids are shapes that you can actually touch....
2833    Animals are made of many cells. They eat thing...
Name: excerpt, Length: 2834, dtype: object

In [12]:
labels=train['target'].tolist()

In [13]:
#texts_test=test['excerpt'].apply(clean_for_readability)

In [14]:
texts_test=test['excerpt']

In [15]:
texts_test

0    My hope lay in Jack's promise that he would ke...
1    Dotty continued to go to Mrs. Gray's every nig...
2    It was a bright and cheerful scene that greete...
3    Cell division is the process by which a parent...
4    Debugging is the process of finding and resolv...
5    To explain transitivity, let us look first at ...
6    Milka and John are playing in the garden. Her ...
Name: excerpt, dtype: object

# сплит

In [16]:
train_texts, eval_texts, train_labels, eval_labels=train_test_split(texts_train, labels, )

# Датасет

In [17]:
class ReadDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len, ):
        self.texts=texts
        self.labels=labels
        self.tokenizer=tokenizer
        self.max_len=max_len
        
    def __len__(self):
        return len(self.texts)
    def __getitem__(self,idx):
        text=self.texts.iloc[idx]
        encoding=self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt',
        )
        return{
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }
    

# модель

In [18]:
MODEL_DIR = "/kaggle/input/huggingface-roberta/"
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR + "roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR + "roberta-base", problem_type='regression', num_labels=1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/huggingface-roberta/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
#name='bert-base-uncased'
#model=AutoModelForSequenceClassification.from_pretrained(name,problem_type='regression', num_labels=1)

# токенайзер

In [20]:
#tokenizer=AutoTokenizer.from_pretrained(name)

# делание датасетов

In [21]:
train_dataset=ReadDataset(train_texts, train_labels, tokenizer, max_len=512)
eval_dataset=ReadDataset(eval_texts, eval_labels, tokenizer, max_len=512)
test_dataset=ReadDataset(texts_test, [0]*len(texts_test), tokenizer, max_len=512)


# Метрика

In [22]:
import numpy as np
from sklearn.metrics import mean_squared_error

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.asarray(preds)
    labels = np.asarray(labels)

    # Частые формы (N, 1) → (N,)
    if preds.ndim == 2 and preds.shape[1] == 1:
        preds = preds.squeeze(1)
    if labels.ndim == 2 and labels.shape[1] == 1:
        labels = labels.squeeze(1)

    # Мультиаутпут: усредняем RMSE по таргетам
    if preds.ndim == 2 and labels.ndim == 2 and preds.shape[1] > 1:
        rmses = [
            mean_squared_error(labels[:, j], preds[:, j], squared=False)
            for j in range(preds.shape[1])
        ]
        return {"rmse": float(np.mean(rmses))}

    rmse = mean_squared_error(labels, preds, squared=False)
    return {"rmse": float(rmse)}


# training args

In [23]:
args=TrainingArguments(
    output_dir='goal',
    learning_rate=3e-5,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    lr_scheduler_type='cosine',
    warmup_ratio=0.05,
    eval_strategy='epoch',
    metric_for_best_model="rmse", 
    greater_is_better=False,
    report_to='none',
    weight_decay=0.01,
)

# trainer

In [24]:
trainer=Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    args=args
)

# обучение

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rmse
1,No log,0.321786,0.567262
2,No log,0.300465,0.548147
3,No log,0.452493,0.672676
4,No log,0.486408,0.697430
5,No log,0.448887,0.669990


TrainOutput(global_step=335, training_loss=0.3185333251953125, metrics={'train_runtime': 773.4814, 'train_samples_per_second': 13.737, 'train_steps_per_second': 0.433, 'total_flos': 2795529863040000.0, 'train_loss': 0.3185333251953125, 'epoch': 5.0})

# сабмит

In [26]:

preds = trainer.predict(test_dataset).predictions.reshape(-1)
sub_simple = pd.DataFrame({"id": test["id"], "target": preds})
sub_simple.to_csv("submission.csv", index=False)

In [27]:
SAVE_DIR = "./roberta_readability_ckpt"   # will hold config.json, pytorch_model.bin, vocab, etc.
trainer.save_model(SAVE_DIR)              # saves model weights + config
tokenizer.save_pretrained(SAVE_DIR)       # saves tokenizer files

# ---------- 4. (optional) zip it for upload ----------
import shutil, os, zipfile
ZIP_NAME = "roberta_readability_ckpt.zip"
shutil.make_archive(base_name=SAVE_DIR, format='zip', root_dir=SAVE_DIR)

print(f"Checkpoint written to {SAVE_DIR}  and zipped as {ZIP_NAME}")

Checkpoint written to ./roberta_readability_ckpt  and zipped as roberta_readability_ckpt.zip
